In [2]:
import pandas as pd
import json

from utils.common_transformers import DTypeTransformer

from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2

from sklearn.feature_selection import RFE
from sklearn.linear_model import Lasso, Ridge

from sklearn.tree import ExtraTreeRegressor

In [3]:
df = pd.read_json("../intermediate_data/basic_fe.json")
df.drop(["date"],axis=1,inplace=True)

In [3]:
df.dtypes

date_month                         int64
client_agent_is_null                bool
date_is_weekend                     bool
checked_out                         bool
date_day_num                       int64
date_quater                        int64
date_close_to_month_start_end       bool
date_day                           int64
date_year                          int64
time_spent                       float64
purchased                           bool
added_in_cart                       bool
date_is_holiday                     bool
date_date_diff                     int64
PC                                  bool
DeviceType                        object
DevicePresLayer                   object
ExpensiveDevice                     bool
agent                             object
dtype: object

In [4]:
X = df.drop("time_spent",axis=1)
y = df["time_spent"]

# Feature Selection

## Common

In [24]:
model = Lasso()
rfe = RFE(model, n_features_to_select=7)
fit = rfe.fit(X, y)
print("Num Features: %s" % (fit.n_features_))
print("Selected Features: %s" % (fit.support_))
print("Feature Ranking: %s" % (fit.ranking_))
selected_features = [feature for index,feature in enumerate(list(X.columns)) if fit.support_[index]]
print(selected_features)

Num Features: 7
Selected Features: [ True  True  True  True False False False False False  True  True  True
 False]
Feature Ranking: [1 1 1 1 3 2 7 5 4 1 1 1 6]
['purchased', 'added_in_cart', 'checked_out', 'client_agent_is_null', 'date_close_to_month_start_end', 'date_is_holiday', 'date_quater']


### Regression

In [35]:
ridge = Ridge(alpha=1.0)
ridge.fit(X,y)
pd.DataFrame({"Feature":list(X.columns),"Coeff":ridge.coef_}).sort_values("Coeff",ascending=False)

,Feature,Coeff
2,checked_out,1125.961960
0,purchased,849.235541
1,added_in_cart,849.048733
5,date_month,85.751368
7,date_day_num,6.202289
12,date_date_diff,-0.158189
4,date_day,-6.442144
8,date_is_weekend,-30.706344
9,date_close_to_month_start_end,-130.938458
10,date_is_holiday,-266.098460


### Classification

In [5]:
test = SelectKBest(score_func=chi2, k=4)
fit = test.fit(X, y)
np.set_printoptions(precision=3)
print(fit.scores_)
print(features[0:5,:])

NameError: name 'y' is not defined

In [36]:
model = ExtraTreeRegressor()
model.fit(X,y)

ExtraTreeRegressor()

In [38]:
pd.DataFrame({"Feature":list(X.columns),"Imp":model.feature_importances_}).sort_values("Imp",ascending=False)

,Feature,Imp
12,date_date_diff,0.208751
4,date_day,0.164078
0,purchased,0.141595
1,added_in_cart,0.132778
5,date_month,0.132156
7,date_day_num,0.078092
11,date_quater,0.049136
8,date_is_weekend,0.034770
2,checked_out,0.027887
9,date_close_to_month_start_end,0.017591
